In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Please excuse the formatting, haven't really done much markdown before. Anyway, we get started by importing some libraries, and loading in our data.


In [0]:
import json
import re
import numpy as np    
import sklearn as sk
import scipy as sp
from sklearn import decomposition
from nltk import PorterStemmer
from scipy import sparse
from sklearn.naive_bayes import BernoulliNB

In [0]:

with open("drive/My Drive/gerald_data/train.json") as file:
  train_data = json.load(file)[::10]
with open("drive/My Drive/gerald_data/test.json") as file:
  test_data = json.load(file)[::10]
with open("drive/My Drive/gerald_data/stopwords.txt") as file:
  stopwords = file.read().splitlines()

# Preprocessing

Like in assignment 2, lowercase this bad boy and remove the punctuation. Then remove stopwords and apply stemming.

In [0]:
ps = PorterStemmer()
def preprocess(post):
  words = re.sub('[^a-z ]', '', post["post"].lower()).split()
  return map(ps.stem, filter(lambda w : w not in stopwords, words))

posts = [preprocess(post) for post in train_data]

Now we create our list of unique words. However, we also want to discard the nonsense, so we delete words from the list that occur in too few posts. 

In [0]:
TOO_RARE_THRESHOLD = 0.0003  # Rate of occurence below which a word is too rare to be considered

words = dict()  # Match words to the number of posts they occured in.
for post in posts:
    for word in list(set(post)): # set trick removes duplicates
       if word not in occured:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1

print "Features before removal of rare words: %d" % len(words)                
for word in words.keys():
    if words[word] < TOO_RARE_THRESHOLD * len(posts):
        del(words[word])
words = sorted(words.keys(), key=words.get, reverse=True)    # words is now a list, sorted by how many times each word occured.    
        
print "After removal: %d" % len(words)

Next we define the functions we will use to vectorize our data, which we can then feed into some models!

In [0]:
def vectorize_x(data):
  X = sp.sparse.lil_matrix((len(data), len(words)), dtype=np.int8)
  for i, post in enumerate(data):
    for word in list(set(post)):
      try: 
        X.rows[i].append(words.index(word)) # In a lil_matrix, there is self.rows, where the ith element is a list of column indexes where there are non-zero elements...
        X.data[i].append(1)                 # ...and self.data, where the ith element is a list of the non-zero elements that occur in that row.
      except: pass
    if i % 1000 == 0:
      print i
  return X.tocsr()
  

def vectorize_gender(data):
  return np.array(map(lambda post : 1 if post["gender"] == "male" else 0, data)) # Change genders to 1:male, 0:female

def vectorize_age(data):
  return np.array((post['age'] for post in data))

Here we apply SVD to reduce the dimensionality of the data to something more managable.

In [27]:
N_COMPONENTS = 300

svd = sk.decomposition.TruncatedSVD(N_COMPONENTS)
X_train = svd.fit_transform(vectorize_x(posts)) 
X_test = svd.transform(vectorize_x([preprocess(post) for post in test_data]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


# Models

In [0]:
Y_gender_train = vectorize_gender(train_data)
Y_gender_test = vectorize_gender(test_data)

bayes = BernoulliNB()
bayes.fit(X_train, Y_gender_train)
print "Naive bayes train accuracy: {:.2%}".format(bayes.score(X_train, Y_gender_train))
print "Naive bayes test accuracy: {:.2%}".format(bayes.score(X_test, Y_gender_test))

N_NEIGHBORS = 10
nn = sk.neighbors.KNeighborsClassifier(N_NEIGHBORS)
nn.fit(X_train, Y_gender_train)
print "Nearest-neighbors train accuracy: {:.2%}".format(nn.score(X_train, Y_gender_train))
print "Nearest-neighbors test accuracy: {:.2%}".format(nn.score(X_test, Y_gender_test))

Naive bayes train accuracy: 61.66%
Naive bayes test accuracy: 59.59%
